In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel

spark = SparkSession.builder.appName("CRMClassifier-Deployment").getOrCreate()
model_path = "../data/rf_model"
data_path = "../data/crm_features.parquet"

df = spark.read.parquet(data_path)
train, test = df.randomSplit([0.8, 0.2], seed=42)

if os.path.exists(model_path):
    rf = RandomForestClassificationModel.load(model_path)
else:
    rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=20).fit(train)
    rf.save(model_path)
    print("✅ Model trained and saved.")

sample = test.limit(5)
predictions = rf.transform(sample)
predictions.select("features", "prediction").show()